In [5]:
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
tree = ET.parse('../Datasets/SUC/suc3.xml')
root = tree.getroot()
ents = ["PRS", "ORG", "LOC", "TME"]
sentsList = []

for sent in root.iter('sentence'):
    prevEntType = None
    tokensList = []
    for child in sent:
        if child.tag == 'name':
            prevEntType = None
            for token in child.iter('w'):
                nameDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(nameDict)
        elif child.tag == 'ne' and child.attrib['type'] in ents and child.attrib['type'] == prevEntType:
            entityTokenList = child.attrib['name'].split(' ')
            for token in child.iter('w'):
                neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|')}
                if token.text == entityTokenList[0]:
                    neDict['ner'] = 'B-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
                else:
                    neDict['ner'] = 'I-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
        elif child.tag == 'ne' and child.attrib['type'] in ents:
            prevEntType = child.attrib['type']
            for token in child.iter('w'):
                neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), 'ner': 'I-' + child.attrib['type']}
                tokensList.append(neDict)
        elif child.tag == 'ne':
            prevEntType = None
            for token in child.iter('w'):
                tokenDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(tokenDict)
        elif child.tag == 'w':
            prevEntType = None
            tokenDict = {'orth': child.text, 'tag': child.attrib['msd'].replace('.', '|'), "ner": "O"}
            tokensList.append(tokenDict)
        else:
            prevEntType = None
            continue
    sentsList.append(tokensList)

In [7]:
train, test = train_test_split(sentsList)

In [8]:
with open('../Datasets/SUC/SUC3trainIOB1.txt', 'w', encoding='utf-8') as trainFile, open('../Datasets/SUC/SUC3testIOB1.txt', 'w', encoding='utf-8') as testFile:
    for sent in train:
        for token in sent:
            trainFile.write(token['orth'] + ' ' + '-' + ' ' + '-' + ' ' + token['ner'] + '\n')
        trainFile.write('\n')
    for sent in test:
        for token in sent:
            testFile.write(token['orth'] + ' ' + token['ner'] + '\n')
        testFile.write('\n')
